#  Experiment 8: Adversarial Debiasing (AIF360) — Full Pipeline
#### Retrain on LendingClub, validate on GermanCredit & GMSC, metrics, SHAP, and saving

In [ ]:


# Step 0: Setup
!pip install aif360 shap pandas matplotlib seaborn scikit-learn tensorflow --quiet

import os
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.metrics import ClassificationMetric

RESULTS_DIR = '/content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation Results _germanCredit &GivemesomeCredit'
os.makedirs(RESULTS_DIR, exist_ok=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.1 MB/s eta 0:00:00


Instructions for updating:
non-resource variables are not supported in the long term
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [ ]:

# -----------------------------
# Step 1: Load & preprocess LendingClub
# -----------------------------
df = pd.read_csv('/content/drive/MyDrive/Research_Thesis_Implementation/data_final/lendingclub_data.csv')
selected_cols = ['loan_status', 'annual_inc', 'term', 'grade', 'home_ownership', 'purpose', 'zip_code']
df = df[selected_cols].dropna()

# Binary target
df['loan_status'] = df['loan_status'].apply(lambda x: 1 if x == 'Fully Paid' else 0)

# Encode categoricals
for col in ['term', 'grade', 'home_ownership', 'purpose', 'zip_code']:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Normalize income
df['annual_inc'] = StandardScaler().fit_transform(df[['annual_inc']])

# AIF360 dataset (protected attribute = zip_code proxy)
privileged_groups = [{'zip_code': 1}]
unprivileged_groups = [{'zip_code': 0}]
aif_data = StandardDataset(df,
                           label_name='loan_status',
                           favorable_classes=[1],
                           protected_attribute_names=['zip_code'],
                           privileged_classes=[[1]])

/tmp/ipython-input-355871729.py:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Research_Thesis_Implementation/data_final/lendingclub_data.csv')


In [ ]:


# -----------------------------
# Step 2: Train Adversarial Debiasing (TF v1)
# -----------------------------
sess = tf.Session()
adversary = AdversarialDebiasing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    scope_name='adv_debiasing',
    debias=True,
    sess=sess
)
adversary.fit(aif_data)

# Predictions on training (for metrics/fairness reference)
preds_train = adversary.predict(aif_data)
y_true = aif_data.labels.ravel()
y_pred = preds_train.labels.ravel()
y_prob = preds_train.scores.ravel()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 10.667402; batch adversarial loss: -156.055954
epoch 1; iter: 0; batch classifier loss: 22.292133; batch adversarial loss: -24.231087
epoch 2; iter: 0; batch classifier loss: 18.013264; batch adversarial loss: -28.180111
epoch 3; iter: 0; batch classifier loss: 11.549299; batch adversarial loss: -47.841148
epoch 4; iter: 0; batch classifier loss: 13.484127; batch adversarial loss: -59.154701
epoch 5; iter: 0; batch classifier loss: 7.731305; batch adversarial loss: -63.950321
epoch 6; iter: 0; batch classifier loss: 6.821599; batch adversarial loss: -86.346298
epoch 7; iter: 0; batch classifier loss: 7.802072; batch adversarial loss: -95.772354
epoch 8; iter: 0; batch classifier loss: 6.298136; batch adversarial loss: -109.508820
epoch 9; iter: 0; batch classifier loss: 4.592690; batch adversarial loss: -134.258362
epoch 10; iter: 0; batch classifier loss: 4.218563; batch adversarial loss: -142.596405
epoch 11; iter: 0; batch classifier loss: 4.

In [ ]:


# -----------------------------
# Step 3: Performance + fairness on LendingClub
# -----------------------------
print("=== LendingClub (Adversarial Debiasing) ===")
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("F1:", f1_score(y_true, y_pred))
print("AUC:", roc_auc_score(y_true, y_prob))

metric_train = ClassificationMetric(aif_data, preds_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
print("SPD:", metric_train.statistical_parity_difference())
print("DI:", metric_train.disparate_impact())
print("EOD:", metric_train.equal_opportunity_difference())
print("AOD:", metric_train.average_odds_difference())
print("BiasAmp:", metric_train.between_group_generalized_entropy_index())
print("Theil:", metric_train.theil_index())

=== LendingClub (Adversarial Debiasing) ===
Accuracy: 0.2748
Precision: 0.8666666666666667
Recall: 0.0035743744844652188
F1: 0.007119386637458927
AUC: 0.5474824554272335
SPD: 0.0
DI: 1.0
EOD: 0.0
AOD: 0.0
BiasAmp: 168.85054564812353
Theil: 1.290816778102604


In [ ]:


# -----------------------------
# Step 4: SHAP (KernelExplainer) for LendingClub
# -----------------------------
# Wrap adversary.predict with a safe model function that accepts numpy and returns scores
def model_fn(x_numpy):
    tmp_df = pd.DataFrame(x_numpy, columns=aif_data.feature_names)
    tmp_df['loan_status'] = 0  # dummy label required by StandardDataset
    ds = StandardDataset(tmp_df,
                         label_name='loan_status',
                         favorable_classes=[1],
                         protected_attribute_names=['zip_code'],
                         privileged_classes=[[1]])
    preds_ds = adversary.predict(ds)
    return preds_ds.scores.ravel()

# Background and evaluation sets (limit for performance)
X_np = aif_data.features
X_df = pd.DataFrame(X_np, columns=aif_data.feature_names)
bg_n = min(100, X_np.shape[0])
eval_n = min(200, X_np.shape[0])

explainer = shap.KernelExplainer(model_fn, X_np[:bg_n])
print("Computing SHAP values for LendingClub...")
sv_train = explainer.shap_values(X_np[:eval_n], nsamples=100)

plt.figure(figsize=(10, 6))
shap.summary_plot(sv_train, X_df.iloc[:eval_n], show=False)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/exp8_lc_shap_global.png', dpi=150, bbox_inches='tight')
plt.close()

Computing SHAP values for LendingClub...


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:


# -----------------------------
# Step 5: Validation on GermanCredit (schema-aligned + fairness + SHAP)
# -----------------------------
GC_PATH = '/content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation dataset/german_credit_data.csv'
df_gc = pd.read_csv(GC_PATH)

# Target mapping
if 'Risk' in df_gc.columns:
    df_gc['loan_status'] = df_gc['Risk'].map({'good': 1, 'bad': 0})
elif 'Creditability' in df_gc.columns:
    df_gc['loan_status'] = df_gc['Creditability']
elif 'class' in df_gc.columns:
    df_gc['loan_status'] = df_gc['class'].map({'good': 1, 'bad': 0})
else:
    raise ValueError("Target column not found in GermanCredit.")

# Encode strings
for col in df_gc.columns:
    if df_gc[col].dtype == 'object':
        df_gc[col] = LabelEncoder().fit_transform(df_gc[col].astype(str))

# Align to LendingClub schema
common_gc = pd.DataFrame()
common_gc['annual_inc']     = df_gc['Credit amount']
common_gc['term']           = df_gc['Duration']
common_gc['grade']          = df_gc['Purpose']
common_gc['home_ownership'] = df_gc['Housing']
common_gc['purpose']        = df_gc['Purpose']
common_gc['zip_code']       = df_gc['Checking account']
common_gc['loan_status']    = df_gc['loan_status']

for col in ['term','grade','home_ownership','purpose','zip_code']:
    common_gc[col] = LabelEncoder().fit_transform(common_gc[col].astype(str))
common_gc['annual_inc'] = StandardScaler().fit_transform(common_gc[['annual_inc']])

# AIF360 dataset for validation
aif_gc = StandardDataset(common_gc,
                         label_name='loan_status',
                         favorable_classes=[1],
                         protected_attribute_names=['zip_code'],
                         privileged_classes=[[1]])

preds_gc = adversary.predict(aif_gc)

y_gc = aif_gc.labels.ravel()
y_pred_gc = preds_gc.labels.ravel()
y_prob_gc = preds_gc.scores.ravel()

print("\n=== GermanCredit Validation (Adversarial Debiasing) ===")
print("Accuracy:", accuracy_score(y_gc, y_pred_gc))
print("Precision:", precision_score(y_gc, y_pred_gc))
print("Recall:", recall_score(y_gc, y_pred_gc))
print("F1:", f1_score(y_gc, y_pred_gc))
print("AUC:", roc_auc_score(y_gc, y_prob_gc))

metric_gc = ClassificationMetric(aif_gc, preds_gc,
                                 unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups)
print("SPD:", metric_gc.statistical_parity_difference())
print("DI:", metric_gc.disparate_impact())
print("EOD:", metric_gc.equal_opportunity_difference())
print("AOD:", metric_gc.average_odds_difference())
print("BiasAmp:", metric_gc.between_group_generalized_entropy_index())
print("Theil:", metric_gc.theil_index())

# SHAP for GC
X_gc_np = aif_gc.features
X_gc_df = pd.DataFrame(X_gc_np, columns=aif_gc.feature_names)
bg_gc = min(100, X_gc_np.shape[0])
eval_gc = min(200, X_gc_np.shape[0])

explainer_gc = shap.KernelExplainer(model_fn, X_gc_np[:bg_gc])
print("Computing SHAP values for GermanCredit...")
sv_gc = explainer_gc.shap_values(X_gc_np[:eval_gc], nsamples=100)

plt.figure(figsize=(10, 6))
shap.summary_plot(sv_gc, X_gc_df.iloc[:eval_gc], show=False)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/exp8_gc_shap_global.png', dpi=150, bbox_inches='tight')
plt.close()


=== GermanCredit Validation (Adversarial Debiasing) ===
Accuracy: 0.319
Precision: 0.620253164556962
Recall: 0.07
F1: 0.1258023106546855
AUC: 0.4944285714285715
SPD: 0.14834613192955798
DI: 3.100268920476373
EOD: 0.15314090191261626
AOD: 0.1538191282049854
BiasAmp: 0.4627324002510242
Theil: 1.07995213678519
Computing SHAP values for GermanCredit...


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:


# -----------------------------
# Step 6: Validation on GiveMeSomeCredit (schema-aligned + fairness + SHAP)
# -----------------------------
GMSC_PATH = '/content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation dataset/GiveMeSomeCredit.csv'
df_gmsc = pd.read_csv(GMSC_PATH)

# Align labels: 1=good (no serious delinquency), 0=bad
df_gmsc['loan_status'] = 1 - df_gmsc['SeriousDlqin2yrs']

for col in df_gmsc.columns:
    if df_gmsc[col].dtype == 'object':
        df_gmsc[col] = LabelEncoder().fit_transform(df_gmsc[col].astype(str))

common_gmsc = pd.DataFrame()
common_gmsc['annual_inc']     = df_gmsc['MonthlyIncome'].fillna(df_gmsc['MonthlyIncome'].median())
common_gmsc['term']           = df_gmsc['NumberOfOpenCreditLinesAndLoans']
common_gmsc['grade']          = df_gmsc['NumberOfTimes90DaysLate']
common_gmsc['home_ownership'] = df_gmsc['NumberRealEstateLoansOrLines']
common_gmsc['purpose']        = df_gmsc['NumberOfTime30-59DaysPastDueNotWorse']
common_gmsc['zip_code']       = df_gmsc['NumberOfDependents'].fillna(0)
common_gmsc['loan_status']    = df_gmsc['loan_status']

for col in ['term','grade','home_ownership','purpose','zip_code']:
    common_gmsc[col] = LabelEncoder().fit_transform(common_gmsc[col].astype(str))
common_gmsc['annual_inc'] = StandardScaler().fit_transform(common_gmsc[['annual_inc']])

# AIF360 dataset for validation
aif_gmsc = StandardDataset(common_gmsc,
                           label_name='loan_status',
                           favorable_classes=[1],
                           protected_attribute_names=['zip_code'],
                           privileged_classes=[[1]])

preds_gmsc = adversary.predict(aif_gmsc)

y_gmsc = aif_gmsc.labels.ravel()
y_pred_gmsc = preds_gmsc.labels.ravel()
y_prob_gmsc = preds_gmsc.scores.ravel()

print("\n=== GiveMeSomeCredit Validation (Adversarial Debiasing) ===")
print("Accuracy:", accuracy_score(y_gmsc, y_pred_gmsc))
print("Precision:", precision_score(y_gmsc, y_pred_gmsc))
print("Recall:", recall_score(y_gmsc, y_pred_gmsc))
print("F1:", f1_score(y_gmsc, y_pred_gmsc))
print("AUC:", roc_auc_score(y_gmsc, y_prob_gmsc))

metric_gmsc = ClassificationMetric(aif_gmsc, preds_gmsc,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)
print("SPD:", metric_gmsc.statistical_parity_difference())
print("DI:", metric_gmsc.disparate_impact())
print("EOD:", metric_gmsc.equal_opportunity_difference())
print("AOD:", metric_gmsc.average_odds_difference())
print("BiasAmp:", metric_gmsc.between_group_generalized_entropy_index())
print("Theil:", metric_gmsc.theil_index())

# SHAP for GMSC
X_gmsc_np = aif_gmsc.features
X_gmsc_df = pd.DataFrame(X_gmsc_np, columns=aif_gmsc.feature_names)
bg_gmsc = min(100, X_gmsc_np.shape[0])
eval_gmsc = min(200, X_gmsc_np.shape[0])

explainer_gmsc = shap.KernelExplainer(model_fn, X_gmsc_np[:bg_gmsc])
print("Computing SHAP values for GiveMeSomeCredit...")
sv_gmsc = explainer_gmsc.shap_values(X_gmsc_np[:eval_gmsc], nsamples=100)

plt.figure(figsize=(10, 6))
shap.summary_plot(sv_gmsc, X_gmsc_df.iloc[:eval_gmsc], show=False)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/exp8_gmsc_shap_global.png', dpi=150, bbox_inches='tight')
plt.close()



=== GiveMeSomeCredit Validation (Adversarial Debiasing) ===
Accuracy: 0.13817333333333334
Precision: 0.902074252217045
Recall: 0.08575163959020961
F1: 0.15661534446764092
AUC: 0.4796559059181351
SPD: 0.130081631577366
DI: 11.218591691313323
EOD: 0.12849486318462255
AOD: 0.14857246620705142
BiasAmp: 0.18801813206123957
Theil: 1.9382285471172258
Computing SHAP values for GiveMeSomeCredit...


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:

# -----------------------------
# Step 7: Save combined results to CSV
# -----------------------------
results_all = pd.DataFrame([
    {
        'Dataset': 'LendingClub(AdvDebias)',
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'AUC': roc_auc_score(y_true, y_prob),
        'SPD': metric_train.statistical_parity_difference(),
        'DI': metric_train.disparate_impact(),
        'EOD': metric_train.equal_opportunity_difference(),
        'AOD': metric_train.average_odds_difference(),
        'BiasAmp': metric_train.between_group_generalized_entropy_index(),
        'Theil': metric_train.theil_index()
    },
    {
        'Dataset': 'GermanCredit(AdvDebias)',
        'Accuracy': accuracy_score(y_gc, y_pred_gc),
        'Precision': precision_score(y_gc, y_pred_gc),
        'Recall': recall_score(y_gc, y_pred_gc),
        'F1': f1_score(y_gc, y_pred_gc),
        'AUC': roc_auc_score(y_gc, y_prob_gc),
        'SPD': metric_gc.statistical_parity_difference(),
        'DI': metric_gc.disparate_impact(),
        'EOD': metric_gc.equal_opportunity_difference(),
        'AOD': metric_gc.average_odds_difference(),
        'BiasAmp': metric_gc.between_group_generalized_entropy_index(),
        'Theil': metric_gc.theil_index()
    },
    {
        'Dataset': 'GiveMeSomeCredit(AdvDebias)',
        'Accuracy': accuracy_score(y_gmsc, y_pred_gmsc),
        'Precision': precision_score(y_gmsc, y_pred_gmsc),
        'Recall': recall_score(y_gmsc, y_pred_gmsc),
        'F1': f1_score(y_gmsc, y_pred_gmsc),
        'AUC': roc_auc_score(y_gmsc, y_prob_gmsc),
        'SPD': metric_gmsc.statistical_parity_difference(),
        'DI': metric_gmsc.disparate_impact(),
        'EOD': metric_gmsc.equal_opportunity_difference(),
        'AOD': metric_gmsc.average_odds_difference(),
        'BiasAmp': metric_gmsc.between_group_generalized_entropy_index(),
        'Theil': metric_gmsc.theil_index()
    }
])

out_csv = f"{RESULTS_DIR}/exp8_adversarial_debiasing_validation_results.csv"
results_all.to_csv(out_csv, index=False)
print(f"\n✅ Results saved to: {out_csv}")
print(f"SHAP plots saved to: {RESULTS_DIR}/exp8_*_shap_global.png")


✅ Results saved to: /content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation Results _germanCredit &GivemesomeCredit/exp8_adversarial_debiasing_validation_results.csv
SHAP plots saved to: /content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation Results _germanCredit &GivemesomeCredit/exp8_*_shap_global.png
